In [2]:
from urllib.request import urlopen,Request 
from bs4 import BeautifulSoup 
import requests
import re
import time
from pathlib import Path
import sys
import os

In [28]:
file = open("./webcrawler/omictool_tools.txt","r")

tool_list = set()
for line in file:
        tool_list.add(line.strip())
print(len(tool_list))

25868


In [40]:
def get_tool():
        try:
                print("Start crawl PMID")
                category_dict = {}
                analytical_dict = {}
                x = 0
                for tool in tool_list:
                    state = tool
                    omictool = "https://omictools.com"+tool.strip()
                    response = requests.get(omictool)
                    html = BeautifulSoup(response.text,"html5lib")
                    
                    parent_category = html.find('div',id='parent-category-container')
                    if parent_category != None:
                        list_of_category = parent_category.find('span').find_all('a',href=True)
                        categories = []
                        for category in list_of_category:
                            category = category['href']
                            if category[len(category)-8:len(category)] == 'category':
                                if category[1:] not in category_dict:
                                    category_dict[category[1:]] = []
                                categories.append(category[1:])

                    analytical_step = html.find('div',id='analytical-step-container')
                    if analytical_step != None:
                        list_of_analytical = analytical_step.find('span').find_all('a',href=True)
                        analyticals = []
                        for category in list_of_analytical:
                            category = category['href']
                            if category[len(category)-8:len(category)] == 'category':
                                if category[1:] not in analytical_dict:
                                    analytical_dict[category[1:]] = []
                                analyticals.append(category[1:])

                    if parent_category != None and analytical_step != None:
                        article = html.find('div',class_='article-reference-infos')
                        if article != None:
                            list_of_pmid = article.find_all_next('div',class_='text',string=True)

                            for pmid in list_of_pmid:
                                if pmid.text.find("PMID") != -1:
                                    for category in categories:
                                        category_dict[category].append(pmid.text.strip()[6:])
                                    for analytical in analyticals:
                                         analytical_dict[analytical].append(pmid.text.strip()[6:])
                    x += 1
                    print("Progress {0:.6f}".format((x /len(tool_list) )*100), end="\r")
                    time.sleep(0.001)
                print("Finsh Crawl PMID")
                return category_dict,analytical_dict
        except:
                print(omictool)
                raise

category_dict,_analytical_dict = get_tool()










Start crawl PMID
Finsh Crawl PMID000


In [33]:
os.mkdir('category')

In [41]:
for category in category_dict:
    w_file = open('./category/'+category+'.txt','a+')
    for pmid in category_dict[category]:
        w_file.write(pmid+'\n')
    w_file.close()

In [42]:
os.mkdir('analytical')

In [45]:
for analytical in _analytical_dict:
    w_file = open('./analytical/'+analytical+'.txt','a+')
    for pmid in _analytical_dict[analytical]:
        w_file.write(pmid+'\n')
    w_file.close()

In [46]:
print(len(_analytical_dict),len(category_dict))

1591 133
